In [25]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

spark = SparkSession.builder.master('local[1]').appName('bbi').getOrCreate()

24/01/10 23:47:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
auto_df_fixed = spark.read.format("csv").option("header", True).load("data/auto-mpg-fixed.csv")
auto_df_fixed.show()
auto_df_fixed.printSchema()

+----+---------+------------+----------+------+------------+---------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|
+----+---------+------------+----------+------+------------+---------+------+--------------------+
|18.0|        8|       307.0|     130.0| 3504.|        12.0|       70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0| 3693.|        11.5|       70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0| 3436.|        11.0|       70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0| 3433.|        12.0|       70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0| 3449.|        10.5|       70|     1|         ford torino|
|15.0|        8|       429.0|     198.0| 4341.|        10.0|       70|     1|    ford galaxie 500|
|14.0|        8|       454.0|     220.0| 4354.|         9.0|       70|     1|    chevrolet impala|
|14.0|    

In [9]:
for (column_name) in ("mpg cylinders displacement horsepower weight acceleration".split()):
    auto_df_fixed = auto_df_fixed.withColumn(column_name, col(column_name).cast("double"))
auto_df_fixed = auto_df_fixed.withColumn("modelyear", col("modelyear").cast("int"))
auto_df_fixed = auto_df_fixed.withColumn("origin", col("origin").cast("int"))

In [15]:
df = auto_df_fixed.withColumn('upper', upper(col('carname'))).show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|               upper|
+----+---------+------------+----------+------+------------+---------+------+--------------------+--------------------+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|       70|     1|chevrolet chevell...|CHEVROLET CHEVELL...|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|       70|     1|   buick skylark 320|   BUICK SKYLARK 320|
|18.0|      8.0|       318.0|     150.0|3436.0|        11.0|       70|     1|  plymouth satellite|  PLYMOUTH SATELLITE|
|16.0|      8.0|       304.0|     150.0|3433.0|        12.0|       70|     1|       amc rebel sst|       AMC REBEL SST|
|17.0|      8.0|       302.0|     140.0|3449.0|        10.5|       70|     1|         ford torino|         FORD TORINO|
|15.0|      8.0|       429.0|     198.0|

In [16]:
auto_df_fixed.withColumn('lower', lower(col('carname'))).show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|               lower|
+----+---------+------------+----------+------+------------+---------+------+--------------------+--------------------+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|       70|     1|chevrolet chevell...|chevrolet chevell...|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|       70|     1|   buick skylark 320|   buick skylark 320|
|18.0|      8.0|       318.0|     150.0|3436.0|        11.0|       70|     1|  plymouth satellite|  plymouth satellite|
|16.0|      8.0|       304.0|     150.0|3433.0|        12.0|       70|     1|       amc rebel sst|       amc rebel sst|
|17.0|      8.0|       302.0|     140.0|3449.0|        10.5|       70|     1|         ford torino|         ford torino|
|15.0|      8.0|       429.0|     198.0|

In [18]:
auto_df_fixed.withColumn('modelyear', concat(lit(19), col('modelyear'))).show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|
+----+---------+------------+----------+------+------------+---------+------+--------------------+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|     1970|     1|chevrolet chevell...|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|     1970|     1|   buick skylark 320|
|18.0|      8.0|       318.0|     150.0|3436.0|        11.0|     1970|     1|  plymouth satellite|
|16.0|      8.0|       304.0|     150.0|3433.0|        12.0|     1970|     1|       amc rebel sst|
|17.0|      8.0|       302.0|     140.0|3449.0|        10.5|     1970|     1|         ford torino|
|15.0|      8.0|       429.0|     198.0|4341.0|        10.0|     1970|     1|    ford galaxie 500|
|14.0|      8.0|       454.0|     220.0|4354.0|         9.0|     1970|     1|    chevrolet impala|
|14.0|    

In [19]:
auto_df_fixed.withColumn('mpg', when(col('mpg') <= 20, 'low').
                        when(col('mpg') <= 30, 'mid').
                         when(col('mpg') <= 40, 'high').
                         otherwise('Very High')).show()
                         

+---+---------+------------+----------+------+------------+---------+------+--------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|
+---+---------+------------+----------+------+------------+---------+------+--------------------+
|low|      8.0|       307.0|     130.0|3504.0|        12.0|       70|     1|chevrolet chevell...|
|low|      8.0|       350.0|     165.0|3693.0|        11.5|       70|     1|   buick skylark 320|
|low|      8.0|       318.0|     150.0|3436.0|        11.0|       70|     1|  plymouth satellite|
|low|      8.0|       304.0|     150.0|3433.0|        12.0|       70|     1|       amc rebel sst|
|low|      8.0|       302.0|     140.0|3449.0|        10.5|       70|     1|         ford torino|
|low|      8.0|       429.0|     198.0|4341.0|        10.0|       70|     1|    ford galaxie 500|
|low|      8.0|       454.0|     220.0|4354.0|         9.0|       70|     1|    chevrolet impala|
|low|      8.0|     

In [20]:
auto_df_fixed.withColumn('xyz', lit(19)).show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+---+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|xyz|
+----+---------+------------+----------+------+------------+---------+------+--------------------+---+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|       70|     1|chevrolet chevell...| 19|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|       70|     1|   buick skylark 320| 19|
|18.0|      8.0|       318.0|     150.0|3436.0|        11.0|       70|     1|  plymouth satellite| 19|
|16.0|      8.0|       304.0|     150.0|3433.0|        12.0|       70|     1|       amc rebel sst| 19|
|17.0|      8.0|       302.0|     140.0|3449.0|        10.5|       70|     1|         ford torino| 19|
|15.0|      8.0|       429.0|     198.0|4341.0|        10.0|       70|     1|    ford galaxie 500| 19|
|14.0|      8.0|       454.0|     220.0|4354.0|         9.0|       70|   